<a href="https://colab.research.google.com/github/hiro106/signate_bento/blob/main/bento_demand_prediction_signate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [1]:
import pandas as pd

In [9]:
pd

<module 'pandas' from '/usr/local/lib/python3.7/dist-packages/pandas/__init__.py'>

In [3]:
import numpy as np

In [10]:
np

<module 'numpy' from '/usr/local/lib/python3.7/dist-packages/numpy/__init__.py'>

In [5]:
from matplotlib import pyplot as plt

In [6]:
plt

<module 'matplotlib.pyplot' from '/usr/local/lib/python3.7/dist-packages/matplotlib/pyplot.py'>

In [7]:
import seaborn as sns

In [8]:
sns

<module 'seaborn' from '/usr/local/lib/python3.7/dist-packages/seaborn/__init__.py'>

In [ ]:
sns.set(font="IPAexGothic", style="white")

In [15]:
train = pd.read_csv("/drive/My Drive/Colab Notebooks/signate/bento/train.csv")

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime       207 non-null    object 
 1   y              207 non-null    int64  
 2   week           207 non-null    object 
 3   soldout        207 non-null    int64  
 4   name           207 non-null    object 
 5   kcal           166 non-null    float64
 6   remarks        21 non-null     object 
 7   event          14 non-null     object 
 8   payday         10 non-null     float64
 9   weather        207 non-null    object 
 10  precipitation  207 non-null    object 
 11  temperature    207 non-null    float64
dtypes: float64(3), int64(2), object(7)
memory usage: 19.5+ KB


In [17]:
test = pd.read_csv("/drive/My Drive/Colab Notebooks/signate/bento/test.csv")

In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   datetime       40 non-null     object 
 1   week           40 non-null     object 
 2   soldout        40 non-null     int64  
 3   name           40 non-null     object 
 4   kcal           36 non-null     float64
 5   remarks        7 non-null      object 
 6   event          3 non-null      object 
 7   payday         2 non-null      float64
 8   weather        40 non-null     object 
 9   precipitation  40 non-null     object 
 10  temperature    40 non-null     float64
dtypes: float64(3), int64(1), object(7)
memory usage: 3.6+ KB


In [24]:
sample = pd.read_csv("/drive/My Drive/Colab Notebooks/signate/bento/sample.csv", header=None)

In [25]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       40 non-null     object
 1   1       40 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 768.0+ bytes


In [26]:
sample

,0,1
0,2014-10-1,24
1,2014-10-2,71
2,2014-10-3,25
3,2014-10-6,32
4,2014-10-7,60
5,2014-10-8,44
6,2014-10-9,64
7,2014-10-10,33
8,2014-10-14,67
9,2014-10-15,28


In [27]:
print("Data Shapes")
print("Train:", train.shape, "Test:", test.shape, "Sample:", sample.shape)

Data Shapes
Train: (207, 12) Test: (40, 11) Sample: (40, 2)
